In [16]:
# Lib imports
import pandas as pd
import cv2
import time
import os

In [3]:
import boto3
s3 = boto3.resource('s3')

# Print out bucket names
listy = []
for bucket in s3.buckets.all():
    listy.append(bucket.name)

In [4]:
listy[1]

'kagstatefarm'

In [5]:
# List everything in bucket
statefarm = s3.Bucket('kagstatefarm')
statefarm_files = []
for file_object in statefarm.objects.all():
    statefarm_files.append(file_object.key)

In [6]:
split_keys = [ [entry.split('/')[0], entry] for entry in statefarm_files]

In [7]:
split_keys_frame =  pd.DataFrame(split_keys, columns = ['folder', 's3_key'])

In [8]:
train_set = split_keys_frame[split_keys_frame.folder == 'train']
test_set = split_keys_frame[split_keys_frame.folder == 'test']

In [17]:
train_breakdown = train_set.s3_key
train_folders = [ [entry.split('/')[1], entry, entry.split('/')[2]] for entry in train_breakdown]
train_frame = pd.DataFrame(train_folders, columns = ['subject', 'key', 'file_name'])

In [18]:
folder_c0 = train_frame[train_frame.subject == 'c0']
folder_c0

,subject,key,file_name
0,c0,train/c0/img_100026.jpg,img_100026.jpg
1,c0,train/c0/img_10003.jpg,img_10003.jpg
2,c0,train/c0/img_100050.jpg,img_100050.jpg
3,c0,train/c0/img_100074.jpg,img_100074.jpg
4,c0,train/c0/img_10012.jpg,img_10012.jpg
5,c0,train/c0/img_100145.jpg,img_100145.jpg
6,c0,train/c0/img_100191.jpg,img_100191.jpg
7,c0,train/c0/img_100257.jpg,img_100257.jpg
8,c0,train/c0/img_100312.jpg,img_100312.jpg
9,c0,train/c0/img_100337.jpg,img_100337.jpg


In [15]:
%ls

Boto_Testing.ipynb      Learning TF.ipynb  README.md           TensorBoard.py
cudnn-70-linux-x64-v40  MNIST/             StateFarm/          tmp/
Extended MNIST.ipynb    MNIST_data/        Tensor_Board.ipynb  Untitled.ipynb


In [37]:
# testing loading and using the dataset
load_dir = 'StateFarm'
for count, row_item in train_frame.iterrows():
    #print row_item[2]
    #key_object = s3.Object('kagstatefarm', row_item[1])
    s3.meta.client.download_file(Bucket = 'kagstatefarm', Key = row_item[1], Filename = os.path.join(load_dir, 'train', row_item[0], row_item[2]))
    # need to use a temp dir
    
    #img = cv2.imread( key_object.download_file() )